<a href="https://colab.research.google.com/github/jiyathart/Citizenai/blob/main/Citizen_AI_%E2%80%93_Intelligent_Citizen_Engagement_Platform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers torch gradio PyPDF2 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.9 MB/s eta 0:00:00


In [4]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import PyPDF2
import io
import requests
from datetime import datetime
import json

# Load model and tokenizer
model_name = "ibm-granite/granite-3.2-2b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def generate_response(prompt, max_length=1024):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)

    if torch.cuda.is_available():
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response.replace(prompt, "").strip()
    return response

def extract_text_from_pdf(pdf_file):
    if pdf_file is None:
        return ""

    try:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
        return text
    except Exception as e:
        return f"Error reading PDF: {str(e)}"

def eco_tips_generator(problem_keywords):
    prompt = f"Generate practical and actionable eco-friendly tips for sustainable living related to: {problem_keywords}. Provide specific solutions and suggestions:"
    return generate_response(prompt, max_length=1000)

def policy_summarization(pdf_file, policy_text):
    # Get text from PDF or direct input
    if pdf_file is not None:
        content = extract_text_from_pdf(pdf_file)
        summary_prompt = f"Summarize the following policy document and extract the most important points, key provisions, and implications for citizens:\n\n{content}"
    else:
        summary_prompt = f"Summarize the following policy document and extract the most important points, key provisions, and implications for citizens:\n\n{policy_text}"

    return generate_response(summary_prompt, max_length=1200)

def community_issue_analysis(issue_description):
    prompt = f"Analyze this community issue from a citizen's perspective: {issue_description}. Provide insights on potential causes, impacts, and suggest actionable solutions that community members and local authorities could implement:"
    return generate_response(prompt, max_length=1000)

def public_service_guide(service_query):
    prompt = f"Create a comprehensive guide for citizens about: {service_query}. Include information on eligibility requirements, application process, required documents, timelines, and contact information for relevant government departments:"
    return generate_response(prompt, max_length=1200)

def civic_engagement_ideas(topic):
    prompt = f"Generate creative ideas for civic engagement and community participation related to: {topic}. Include suggestions for both individual actions and community initiatives that can drive positive change:"
    return generate_response(prompt, max_length=1000)

def government_feedback_analysis(feedback_text):
    prompt = f"Analyze this citizen feedback for a government service: {feedback_text}. Identify key sentiments, suggestions for improvement, and potential actionable items for government authorities:"
    return generate_response(prompt, max_length=1000)

# Create enhanced Gradio interface for Citizen AI
with gr.Blocks(theme=gr.themes.Soft(), title="Citizen AI – Intelligent Citizen Engagement Platform") as app:
    gr.Markdown("# 🏛️ Citizen AI – Intelligent Citizen Engagement Platform")
    gr.Markdown("### Empowering citizens through AI-driven insights and engagement tools")

    with gr.Tabs():
        with gr.TabItem("🌱 Sustainable Living"):
            with gr.Row():
                with gr.Column():
                    keywords_input = gr.Textbox(
                        label="Environmental Concern or Topic",
                        placeholder="e.g., plastic reduction, energy saving, water conservation...",
                        lines=3
                    )
                    generate_tips_btn = gr.Button("Generate Sustainable Solutions", variant="primary")

                with gr.Column():
                    tips_output = gr.Textbox(label="Actionable Tips & Solutions", lines=15)

            generate_tips_btn.click(eco_tips_generator, inputs=keywords_input, outputs=tips_output)

        with gr.TabItem("📋 Policy Analysis"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### Analyze Government Policies & Documents")
                    pdf_upload = gr.File(label="Upload Policy PDF", file_types=[".pdf"])
                    policy_text_input = gr.Textbox(
                        label="Or paste policy text here",
                        placeholder="Paste policy document text...",
                        lines=5
                    )
                    summarize_btn = gr.Button("Analyze Policy", variant="primary")

                with gr.Column():
                    summary_output = gr.Textbox(label="Policy Summary & Citizen Implications", lines=20)

            summarize_btn.click(policy_summarization, inputs=[pdf_upload, policy_text_input], outputs=summary_output)

        with gr.TabItem("🏘️ Community Issues"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### Community Problem Solver")
                    issue_input = gr.Textbox(
                        label="Describe a community issue or challenge",
                        placeholder="e.g., poor waste management in neighborhood, traffic congestion, lack of green spaces...",
                        lines=5
                    )
                    analyze_issue_btn = gr.Button("Analyze Issue & Suggest Solutions", variant="primary")

                with gr.Column():
                    issue_output = gr.Textbox(label="Issue Analysis & Proposed Solutions", lines=20)

            analyze_issue_btn.click(community_issue_analysis, inputs=issue_input, outputs=issue_output)

        with gr.TabItem("ℹ️ Public Services Guide"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### Public Services Information")
                    service_input = gr.Textbox(
                        label="What government service do you need information about?",
                        placeholder="e.g., applying for a passport, property registration, business license...",
                        lines=3
                    )
                    service_guide_btn = gr.Button("Generate Service Guide", variant="primary")

                with gr.Column():
                    service_output = gr.Textbox(label="Comprehensive Service Guide", lines=20)

            service_guide_btn.click(public_service_guide, inputs=service_input, outputs=service_output)

        with gr.TabItem("🤝 Civic Engagement"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### Civic Participation Ideas")
                    engagement_input = gr.Textbox(
                        label="What community or civic topic interests you?",
                        placeholder="e.g., neighborhood safety, community gardens, local governance...",
                        lines=3
                    )
                    engagement_btn = gr.Button("Generate Engagement Ideas", variant="primary")

                with gr.Column():
                    engagement_output = gr.Textbox(label="Civic Engagement Ideas", lines=20)

            engagement_btn.click(civic_engagement_ideas, inputs=engagement_input, outputs=engagement_output)

        with gr.TabItem("📝 Feedback Analysis"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### Government Service Feedback Analyzer")
                    feedback_input = gr.Textbox(
                        label="Paste your feedback about a government service",
                        placeholder="Describe your experience with a public service...",
                        lines=5
                    )
                    feedback_btn = gr.Button("Analyze Feedback", variant="primary")

                with gr.Column():
                    feedback_output = gr.Textbox(label="Feedback Analysis & Recommendations", lines=20)

            feedback_btn.click(government_feedback_analysis, inputs=feedback_input, outputs=feedback_output)

    gr.Markdown("---")
    gr.Markdown("### 📞 Need human assistance? Contact your local government office during business hours.")
    gr.Markdown("### ⚠️ Disclaimer: This AI assistant provides informational guidance only. Always verify critical information with official sources.")

app.launch(share=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://11a044d0a7a510d9ce.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
